In [1]:
import numpy as np
import pandas as pd
import ast
import itertools
import collections
import struct
import sys
from sympy import poly, var
from collections import Counter

In [2]:
col_names  = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']
row_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

enc_key_sbox = pd.read_excel('AES tables.xlsx', index_col=0)

enc_key_sbox.columns = col_names ## replacing the column names 
enc_key_sbox.index = row_names    ## replacing the row names

enc_key_sbox.head()

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
0,63,7c,77,7b,f2,6b,6f,c5,30,1,67,2b,fe,d7,ab,76
1,ca,82,c9,7d,fa,59,47,f0,ad,d4,a2,af,9c,a4,72,c0
2,b7,fd,93,26,36,3f,f7,cc,34,a5,e5,f1,71,d8,31,15
3,4,c7,23,c3,18,96,5,9a,7,12,80,e2,eb,27,b2,75
4,9,83,2c,1a,1b,6e,5a,a0,52,3b,d6,b3,29,e3,2f,84


## Key Generation

In [3]:
def key_sbox(element):
    row_index = element[0]
    col_index  = element[1]
    ans = enc_key_sbox.loc[[row_index],[col_index]][col_index][0]
    return str(ans)

In [4]:
def binaryToDecimal(binary):
    binary1 = binary
    decimal, i, n = 0, 0, 0
    
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
        
    hexadecimal =  hex(decimal)[-1]
    return hexadecimal

In [5]:
def bcd_hexadecimal(bcd_list):
    w4 = []
    
    for i in range(0,len(bcd_list),2):
        temp_str = ' '
        temp_str = binaryToDecimal(int(bcd_list[i]))
        temp_str = temp_str + binaryToDecimal(int(bcd_list[i+1]))
        w4.append(temp_str)
        
    return w4

In [6]:
def HexaDecimaltoBCD(str):
    list1 = []
    for i in range(len(str)):
        decimal = int(str[i], 16)                                ## hexadecimal -> decimal
        binary_num = bin(decimal).replace("0b", "")  # decimal -> binary
        list1.append(binary_num)
            
    ## binary in-terms of 4 bits
    for i in range(len(list1)):
        element = list1[i]
        if len(element)<4:
            diff = 4 - len(element)
            for j in range(diff):
                element = "0" + element
            list1[i] = element
            
    return list1

In [7]:
def Hexaword_BCD(list1):
    bcd = [ HexaDecimaltoBCD(i) for i in list1 ]
    bcd = list(np.concatenate(bcd).flat)    ## 2d list to 1d list
    return bcd

In [8]:
def XOR(list1, list2):
    
    def compare(element1, element2):
        ans = []
        for i,j in zip(element1,element2):
            if (i!=j):ans.append(1)
            else:ans.append(0)
        return ans

    main_ans = []
    for i in range(len(list1)):
        main_ans.append(compare(list1[i], list2[i]))
        
    main_ans = [ "".join(list(map(str, i))) for i in main_ans ]
    return main_ans

In [9]:
def binary_to_polynomial(a):
    str1 = ""
    nobits = len(a)
    for x in range (0,nobits-2):
        if (a[x] == '1'):
            if (len(str1)==0):str1 +="x^"+str(nobits-x-1)
            else: str1 +=" + x^"+str(nobits-x-1)
    if (a[nobits-2] == '1'):
        if (len(str1)==0):str1 +="x"
        else:str1 +=" + x"

    if (a[nobits-1] == '1'):str1 +="+1"
    print(str1)

In [10]:
def binary_division_modulo_2(val1, val2):
    
    def xor(a, b):
        result = []
        for i in range(1, len(b)):
            if a[i] == b[i]:result.append('0')
            else:result.append('1')
        return ''.join(result)

    def showpoly(a):
        str1 = ""
        nobits = len(a)
        for x in range (0,nobits-2):
            if (a[x] == '1'):
                if (len(str1)==0):str1 +="x**"+str(nobits-x-1)	
                else: str1 +="+x**"+str(nobits-x-1)
        if (a[nobits-2] == '1'):
            if (len(str1)==0):str1 +="x"
            else:str1 +="+x"

        if (a[nobits-1] == '1'):str1 +="+1"
        print(str1)
        
    def divide(dividend, divisor):
        pick = len(divisor)
        tmp = dividend[0 : pick]

        while (pick < len(dividend)):

            if tmp[0] == '1':tmp = xor(divisor, tmp) + dividend[pick]
            else: tmp = xor('0'*pick, tmp) + dividend[pick]
            pick += 1

        if tmp[0] == '1':tmp = xor(divisor, tmp)
        else:tmp = xor('0'*pick, tmp)

        checkword = tmp
        return checkword

    val1= divide(val1,val2)
    return val1

In [11]:
def bcd_to_hexadecimal(list1):
        hexadecimal_ans = bcdToHexaDecimal(list1)
        hexadecimal_ans.reverse()   ## reversing the list
        hexadecimal_ans = [ i.lower() for i in hexadecimal_ans]  ## upper-case alphabets to lower-case
        hexadecimal_ans = ''.join(hexadecimal_ans)
        return hexadecimal_ans

In [12]:
# Function to convert BCD to hexadecimal
def bcdToHexaDecimal(s):
    len1 = len(s)
    check = 0
    num = 0
    sum = 0
    mul = 1
    ans = []

# Iterating through the bits backwards
    i = len1 - 1
     
    while(i >= 0):
        sum += (ord(s[i]) - ord('0')) * mul
        mul *= 2
        check += 1
 
        # Computing the hexadecimal number formed
        # so far and storing it in a vector.
        if (check == 4 or i == 0):
             
            if (sum <= 9):
                ans.append(chr(sum + ord('0')))
            else:
                ans.append(chr(sum + 55));
 
            # Reinitializing all variables for next group.
            check = 0
            sum = 0
            mul = 1
        i -= 1
         
    len1 = len(ans)
 
    # Printing the hexadecimal
    # number formed so far.
    i = len1 - 1
     
    while(i >= 0):
        return ans

In [13]:
def rotword(word):
    rot=word[1:]
    rot.append(word[0])
    return rot

In [14]:
def col1_generation(iteration_var, hex_key):
    
    ## round-constant table 
    ## key-round and value-hexadecimal
    round_constant = {1:'1', 2:'2', 3:'4', 4:'8', 5:'10', 6:'20', 7:'40', 8:'80', 9:'1B', 10:'36'}

    ## taking the last column words
    last_col = hex_key[-1]

    ## left shifting the last column words
    #left_shift = last_col[1:] + last_col[:1] 
    left_shift = rotword(last_col)

    ## sub-word generation from S-Box
    subword = []
    for i in left_shift:
        val = key_sbox(i)
        if len(val)!=2:
            val = '0' + val
            subword.append(val)
        else:
            subword.append(val)

    ## subword --> hexadecimal(subword)
    y1 = Hexaword_BCD(subword)

    ## subword (XOR) Round Constant
    element = round_constant[iteration_var]
    initial = HexaDecimaltoBCD(element)
    initial_length = len(initial)

    if (initial_length == 1):
        between = initial
        temp_1 = [ [0 for j in range(4) for i in range(1)] ]
        last = [ [0 for j in range(4)] for i in range(6) ]
        temp_1.extend(between)
        temp_1.extend(last)
        final = temp_1
        if (element==1): ## if there is a single element(0,1,2, ... ,9) from the s-box --> length=1
            before1 = [ [0 for j in range(4) for i in range(1)] ]
            before1.extend(final)
            final = before1

    elif (initial_length == 2):  ## if there is a two element(11, 1a, b1, ...) from the s-box --> length=2
        last = [ [0 for j in range(4)] for i in range(6) ]
        first = initial
        first.extend(last)
        final = first

    final = [ "".join(list(map(str, i))) for i in final ]
    round_list = final

    round_ans = XOR(y1, round_list) ## --> g(col4_before)
    #print("y1 : ",y1)
    # taking col1_before
    col1_before = [ HexaDecimaltoBCD(i) for i in hex_key[0] ]
    col1_before = list(np.concatenate(col1_before).flat)

    ## col1 --> col1_before (exor) g(col4_before)
    col1 = XOR(col1_before, round_ans)
    col1 = bcd_hexadecimal(col1)

    return col1

In [15]:
def key_generation(hex_key):
    complete_keys = []
    for i in range(1,11): ## 10 times running
        temp = []

        ## 1st col word = col_before[0] (exor) g(col_before[-1]) 
        col1_hex = col1_generation(i, hex_key)
        col1_bin  = Hexaword_BCD(col1_hex)

        for j in range(2,5): ## each loop, 3 times running ( remaining 3 words)
            if (j==2):  ## 2nd col words
                col1_before_bin = Hexaword_BCD(hex_key[j-1])
                col2_bin = XOR(col1_before_bin, col1_bin)
                col2_hex = bcd_hexadecimal(col2_bin)

            else:  ## 3rd and 4th column words
                col2_before_bin = Hexaword_BCD(hex_key[j-1])            
                col3_bin = XOR(col2_before_bin, col2_bin)
                col3_hex = bcd_hexadecimal(col3_bin)
                col2_bin = Hexaword_BCD(col3_hex)
                temp.append(col3_hex)

        ## once again generating the hex_key
        hex_key = []

        hex_key.append(col1_hex)
        hex_key.append(col2_hex)
        hex_key.extend(temp)    
        hex_key = pd.DataFrame(hex_key).T.values.tolist()

        complete_keys.append(hex_key)
    return complete_keys

## Round Operation

In [16]:
def add_round_key(hex_str, complete_keys,round_num): ## exor with plain text and key
    add_round_key = []
    for i in range(4):
        bin_str = Hexaword_BCD(hex_str[i])
        bin_keys = Hexaword_BCD(complete_keys[round_num][i])
        bin_xor   = XOR(bin_str, bin_keys)
        bcd_round_key = bcd_hexadecimal(bin_xor)
        add_round_key.append(bcd_round_key)
    return pd.DataFrame(add_round_key).T.values.tolist()

## Substitution Box

In [17]:
def subtitute_box(add_round_key):  ## output from add_round_key
    subword = []
    for i in range(4):
        temp_word = []
        for j in add_round_key:
            val = key_sbox(j[i])
            if len(val)!=2:
                val = '0' + val
                temp_word.append(val)
            else:
                temp_word.append(val)

        ## subword --> hexadecimal(subword)
        temp_word = Hexaword_BCD(temp_word)
        temp_word_hexa = bcd_hexadecimal(temp_word)
        subword.append(temp_word_hexa)
        temp_word_hexa = []

    ## swapping rows and columns
    #subword = [list(x) for x in zip(*subword)]
    return subword

## Shift Rows

In [18]:
def shift_rows(subtitute_box_ans):
    ## shifting rows and columns
    shift_rows = []
    subword = collections.deque(subtitute_box_ans)
    for i in range(4):
        temp = collections.deque(subtitute_box_ans[i])
        temp.rotate(-i)
        shift_rows.append(list(temp))
    return shift_rows

## Mix Columns

In [19]:
def mix_columns_each_round(shift_rows):
    
    multiple = [['02', '03', '01', '01'],
                 ['01', '02', '03', '01'],
                 ['01', '01', '02', '03'],
                 ['03', '01', '01', '02']]

    prod_ans1 = []
    row_ans = []
    final_ans  = []

    for i in range(4):
        for j in range(4):
            for k in range(4):

                ## hexadecimal to BCD
                num1 = Hexaword_BCD(multiple[i][k])
                num2 = Hexaword_BCD(shift_rows[j][k])
                #print("\nHexadecimal Format : num-1 : {} and num-2 : {} ".format(multiple[i][k], shift_rows[j][k]))
                #print("BCD Format : num-1 : {} and num-2 : {} ".format(num1, num2))

                ## Product operation
                num1_list = [int(item) for sublist in num1 for item in sublist]
                num2_list = [int(item) for sublist in num2 for item in sublist]
                ans = list(np.poly1d(num1_list) * np.poly1d(num2_list))
                prod_ans1.append(ans)

            max_length = max([len(p) for p in prod_ans1])

           ## adding 0's 
            temp_match_len = []
            for a in prod_ans1:
                for b in range(max_length - len(a)):
                    a.insert(0, 0)
                element = ''.join(map(str, a))
                #binary_to_polynomial(element)
                temp_match_len.append(a)
            prod_ans1 = temp_match_len

            ## summing up the polynomials
            temp_sum = []
            for c in range(max_length):
                temp_sum.append(sum([sub[c] for sub in prod_ans1]))
            for d in range(len(temp_sum)):
                if (temp_sum[d] %2!= 0): temp_sum[d] = 1  ## sum is odd number put 1
                else:  temp_sum[d] = 0                               ## sum is even number put 0
            prod_ans1 = temp_sum

            prod_ans1= ''.join(map(str, prod_ans1))
           #binary_to_polynomial(prod_ans1)

            if len(prod_ans1)>8:
                irreducible_polynomial = '100011011'
                ir_ans = binary_division_modulo_2(prod_ans1, irreducible_polynomial)
                prod_ans1= ir_ans
                hexadecimal_ans = bcd_to_hexadecimal(prod_ans1)
                row_ans.append(hexadecimal_ans)
            else:
                hexadecimal_ans = bcd_to_hexadecimal(prod_ans1)
                row_ans.append(hexadecimal_ans)
                
            prod_ans1 = []
        final_ans.append(row_ans)
        row_ans = []
    return final_ans

In [20]:
def main():
    round_num = 0
    add_round_key
    subtitute_box
    shift_rows
    mix_columns_each_round

## Getting the user inputs

In [21]:
def text_hexadecimal(text): ## all the blocks --> 16bytes
    hex_text   = []
    for i in text: ## character -> ascii (decimal) -> hexa-decimal
        hex_text.append(hex(ord(i))[2:])
    return hex_text ## 16-byte representation of the text

In [22]:
plain_text = "Two One Nine Two"
plain_key = 'Thats my Kung Fu' 

hex_str = text_hexadecimal(plain_text)
hex_key = text_hexadecimal(plain_key)

print("Text Hexadecimal Representation : ",hex_str)
print("Key Hexadecimal Representation : ",hex_key)

Text Hexadecimal Representation :  ['54', '77', '6f', '20', '4f', '6e', '65', '20', '4e', '69', '6e', '65', '20', '54', '77', '6f']
Key Hexadecimal Representation :  ['54', '68', '61', '74', '73', '20', '6d', '79', '20', '4b', '75', '6e', '67', '20', '46', '75']


## Key Generation

In [23]:
## splitting into 4*4 matrix
hex_key = [hex_key[i:i+4] for i in range(0, len(hex_key), 4)]
hex_key

[['54', '68', '61', '74'],
 ['73', '20', '6d', '79'],
 ['20', '4b', '75', '6e'],
 ['67', '20', '46', '75']]

In [24]:
r0_key = [] ## to accomodate Round-0th key
r0_key.extend(hex_key)
r0_key

[['54', '68', '61', '74'],
 ['73', '20', '6d', '79'],
 ['20', '4b', '75', '6e'],
 ['67', '20', '46', '75']]

In [25]:
r1_r10_keys = key_generation(hex_key)
r1_r10_keys.insert(0,(r0_key))
complete_keys = r1_r10_keys

for i in range(len(complete_keys)):
    print("Round - {} keys --> ".format(i),end=' ')
    print(*list(itertools.chain(*complete_keys[i])))

Round - 0 keys -->  54 68 61 74 73 20 6d 79 20 4b 75 6e 67 20 46 75
Round - 1 keys -->  e2 91 b1 d6 32 12 59 79 fc 91 e4 a2 f1 88 e6 93
Round - 2 keys -->  24 16 ea 1b 1f 0d 9c 14 6d 34 d0 36 77 0e ac 3f
Round - 3 keys -->  8b 94 f9 8e 87 8a be b0 9f 03 d3 7f ee fa cc f3
Round - 4 keys -->  ae 29 b6 58 df 55 56 ac f4 4a 99 55 a6 16 69 9a
Round - 5 keys -->  f9 26 d2 74 d0 85 cf d9 0e 58 c1 a8 7c d0 85 1f
Round - 6 keys -->  a9 79 77 0b b1 34 6c bc 12 dd 1c 99 64 bd 15 0a
Round - 7 keys -->  93 22 30 54 20 14 c9 74 10 7c 60 75 48 f4 6d 67
Round - 8 keys -->  ac 8c 9c d4 1e 0a 76 82 b5 7c 1c 71 06 72 07 60
Round - 9 keys -->  f7 e9 5c 5a 49 43 3f 4d 4c 3a 26 21 bb 39 48 28
Round - 10 keys -->  d3 9a d6 6d bb f8 c2 fb 68 57 71 39 b0 fd dc f4


## Making the string ready

In [26]:
hex_str = [hex_str[i:i+4] for i in range(0, len(hex_str), 4)]
print("Hexadecimal Representation : ",hex_str)

Hexadecimal Representation :  [['54', '77', '6f', '20'], ['4f', '6e', '65', '20'], ['4e', '69', '6e', '65'], ['20', '54', '77', '6f']]


## Main Function

## Round-1

In [27]:
rnd_cnt = 0
add_round_key_ans = add_round_key(hex_str, complete_keys,rnd_cnt)
add_round_key_ans

[['00', '3c', '6e', '47'],
 ['1f', '4e', '22', '74'],
 ['0e', '08', '1b', '31'],
 ['54', '59', '0b', '1a']]

In [28]:
subtitute_box_ans = subtitute_box(add_round_key_ans)
shift_rows_ans = [['63', '2f', 'af', 'a2'],
                            ['eb', '93', 'c7', '20'],
                            ['9f', '92', 'ab', 'cb'],
                            ['a0', 'c0', '30', '2b']]
mix_columns_ans = mix_columns_each_round(shift_rows_ans)

In [29]:
mix_columns_ans

[['ba', '84', 'e8', '1b'],
 ['75', 'a4', '8d', '40'],
 ['f4', '8d', '06', '7d'],
 ['7a', '32', '0e', '5d']]

In [30]:
complete_keys[1]

[['e2', '91', 'b1', 'd6'],
 ['32', '12', '59', '79'],
 ['fc', '91', 'e4', 'a2'],
 ['f1', '88', 'e6', '93']]

In [31]:
rnd_cnt = 1
add_round_key_ans = add_round_key(mix_columns_ans, complete_keys,rnd_cnt)

#add_round_key_ans = pd.DataFrame(add_round_key_ans).T.values.tolist()
print("Add Round Key : ",add_round_key_ans)

Add Round Key :  [['58', '47', '08', '8b'], ['15', 'b6', '1c', 'ba'], ['59', 'd4', 'e2', 'e8'], ['cd', '39', 'df', 'ce']]


## Round-2 to 9

In [32]:
for i in range(2,10):  ## from Round-1 to Round-2
    print("\n")
    rnd_cnt = i
    subtitute_box_ans = subtitute_box(add_round_key_ans)
    
    shift_rows_ans     = shift_rows(subtitute_box_ans)
    shift_rows_ans = pd.DataFrame(shift_rows_ans).T.values.tolist()
    
    mix_columns_ans = mix_columns_each_round(shift_rows_ans)
    add_round_key_ans = add_round_key(mix_columns_ans, complete_keys, rnd_cnt)
    
    print("Sibstitute Box : ",subtitute_box_ans)
    print("Shift Rows      : ",shift_rows_ans)
    print("Mix Columns   : ",mix_columns_ans)
    print("Keys               : ",complete_keys[rnd_cnt])
    print("Add Round Key : ",add_round_key_ans)



Sibstitute Box :  [['6a', '59', 'cb', 'bd'], ['a0', '4e', '48', '12'], ['30', '9c', '98', '9e'], ['3d', 'f4', '9b', '8b']]
Shift Rows      :  [['6a', '4e', '98', '8b'], ['59', '48', '9e', '3d'], ['cb', '12', '30', 'f4'], ['bd', 'a0', '9c', '9b']]
Mix Columns   :  [['15', 'c9', '7f', '9d'], ['ce', '4d', '4b', 'c2'], ['89', '71', 'be', '88'], ['65', '47', '97', 'cd']]
Keys               :  [['24', '16', 'ea', '1b'], ['1f', '0d', '9c', '14'], ['6d', '34', 'd0', '36'], ['77', '0e', 'ac', '3f']]
Add Round Key :  [['31', 'd1', 'e4', '12'], ['df', '40', '45', '49'], ['95', 'd7', '6e', '3b'], ['86', 'd6', 'be', 'f2']]


Sibstitute Box :  [['c7', '9e', '2a', '44'], ['3e', '09', '0e', 'f6'], ['69', '6e', '9f', 'ae'], ['c9', '3b', 'e2', '89']]
Shift Rows      :  [['c7', '09', '9f', '89'], ['9e', '0e', 'ae', 'c9'], ['2a', 'f6', '69', '3b'], ['44', '3e', '6e', 'e2']]
Mix Columns   :  [['98', '52', '07', '46'], ['e6', 'a2', '5d', '68'], ['6b', '97', '43', '9b'], ['cd', '90', '97', '43']]
Keys     

## Round-10

In [33]:
rnd_cnt = 10

subtitute_box_ans = subtitute_box(add_round_key_ans)
shift_rows_ans = shift_rows(subtitute_box_ans)
add_round_key_ans = add_round_key(hex_str, complete_keys,rnd_cnt)
print("Sibstitute Box : ",subtitute_box_ans)
print("Shift Rows      : ",shift_rows_ans)
print("Add Round Key : ",add_round_key_ans)

Sibstitute Box :  [['e0', '16', '0e', '31'], ['68', 'f2', '31', '06'], ['18', '87', '0a', '20'], ['05', 'b9', '01', '86']]
Shift Rows      :  [['e0', '16', '0e', '31'], ['f2', '31', '06', '68'], ['0a', '20', '18', '87'], ['86', '05', 'b9', '01']]
Add Round Key :  [['87', 'f4', '26', '90'], ['ed', '96', '3e', 'a9'], ['b9', 'a7', '1f', 'ab'], ['4d', 'db', '5c', '9b']]
